Scope: Prediction performance on crossed expertise

In [ ]:
import pickle   
import numpy as np
from scipy.io import savemat
import pandas as pd 
from sklearn import metrics

from tensorflow.keras.models import load_model 
from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


List of decision timescale (NameHorizon), sampling time (NameStep) and expertise (NameExpertiseAndStep) to be used to loop over trained ANN models and computed cross performance

In [ ]:
NameHorizon = ['1', '8', '16', '32']

NameStep = ['step1', 'step2', 'step4','step1', 'step2', 'step4']

NameExertiseAndStep = ['Novice_step1', 'Novice_step2', 'Novice_step4', 
                      'Expert_step1', 'Expert_step2', 'Expert_step4']

List of ANN trained and samples used for each combination of sampling time, decision horizon and expertise

In [ ]:
directory = './checkpoint/FinalModels/'

Novice_step1_set = [directory + '15032021/TrainTestSets_reduced3_indexes_001', 
                directory + '11032021/TrainTestSets_reduced3_indexes_004',
                directory + '13032021/TrainTestSets_reduced3_indexes_001',
                directory + '14032021/TrainTestSets_reduced3_indexes_001']

Novice_step2_set = [directory + '11032021/TrainTestSets_reduced3_indexes_002', 
                directory + '11032021/TrainTestSets_reduced3_indexes_005',
                directory + '13032021/TrainTestSets_reduced3_indexes_002',
                directory + '14032021/TrainTestSets_reduced3_indexes_002']

Novice_step4_set = [directory + '11032021/TrainTestSets_reduced3_indexes_003', 
                directory + '13032021/TrainTestSets_reduced3_indexes_008',
                directory + '13032021/TrainTestSets_reduced3_indexes_003',
                directory + '14032021/TrainTestSets_reduced3_indexes_003']

Expert_step1_set = [directory + '12032021/TrainTestSets_reduced3_indexes_001', 
                directory + '12032021/TrainTestSets_reduced3_indexes_004',
                directory + '13032021/TrainTestSets_reduced3_indexes_004',
                directory + '14032021/TrainTestSets_reduced3_indexes_004']

Expert_step2_set = [directory + '12032021/TrainTestSets_reduced3_indexes_002', 
                directory + '12032021/TrainTestSets_reduced3_indexes_005',
                directory + '13032021/TrainTestSets_reduced3_indexes_005',
                directory + '14032021/TrainTestSets_reduced3_indexes_005']

Expert_step4_set = [directory + '12032021/TrainTestSets_reduced3_indexes_003', 
                directory + '13032021/TrainTestSets_reduced3_indexes_007',
                directory + '13032021/TrainTestSets_reduced3_indexes_006',
                directory + '14032021/TrainTestSets_reduced3_indexes_006']

file_set_list = [Novice_step1_set, Novice_step2_set, Novice_step4_set, Expert_step1_set, Expert_step2_set, Expert_step4_set]

Novice_step1_model = [directory + '15032021/15032021001_reduced3', 
            directory + '11032021/11032021004_reduced3',
            directory + '13032021/13032021001_reduced3',
            directory + '14032021/14032021001_reduced3']

Novice_step2_model = [directory + '11032021/11032021002_reduced3', 
                directory + '11032021/11032021005_reduced3',
                directory + '13032021/13032021002_reduced3',
                directory + '14032021/14032021002_reduced3']

Novice_step4_model = [directory + '11032021/11032021003_reduced3', 
                directory + '13032021/13032021008_reduced3',
                directory + '13032021/13032021003_reduced3',
                directory + '14032021/14032021003_reduced3']

Expert_step1_model = [directory + '12032021/12032021001_reduced3', 
                directory + '12032021/12032021004_reduced3',
                directory + '13032021/13032021004_reduced3',
                directory + '14032021/14032021004_reduced3']

Expert_step2_model = [directory + '12032021/12032021002_reduced3', 
                directory + '12032021/12032021005_reduced3',
                directory + '13032021/13032021005_reduced3',
                directory + '14032021/14032021005_reduced3']

Expert_step4_model = [directory + '12032021/12032021003_reduced3', 
                directory + '13032021/13032021007_reduced3',
                directory + '13032021/13032021006_reduced3',
                directory + '14032021/14032021006_reduced3']

file_model_list = [Novice_step1_model, Novice_step2_model, Novice_step4_model, 
                   Expert_step1_model, Expert_step2_model, Expert_step4_model]


file_sequences_directory = './dataset/sequences/'

In [ ]:
key_horizon = 0
# key_expstep = 0
# key_test = 0

In [ ]:
# Debugging cell
# NameExertiseAndStep[key_expstep] , NameExertiseAndStep[key_test] , NameStep[key_expstep], NameStep[key_test]

In [ ]:
for key_horizon in range(len(NameHorizon)):
    for key_expstep in range(len(NameExertiseAndStep)):
        for key_test in range(len(NameExertiseAndStep)):
            
            # Check if sampling time and decision horizon of training and test set coincide
            # otherwise skip forward
            if NameStep[key_expstep] == NameStep[key_test]:

                print('\n')
                print(NameExertiseAndStep[key_expstep] , NameExertiseAndStep[key_test] , NameStep[key_expstep], 
                      NameStep[key_test])

                # load the model 
                file_to_open = file_set_list[key_expstep][key_horizon]
                file_model = file_model_list[key_expstep][key_horizon]

                #load training and test samples
                file_sequences = file_sequences_directory + 'SequencesTargets_'+ NameExertiseAndStep[key_test]+'_'+NameHorizon[key_horizon]


                SeqTarList_file = open(file_sequences,"rb")
                SeqTarList = pickle.load(SeqTarList_file)
                SeqTarList_file.close()

                sequences = SeqTarList[0]
                targets = SeqTarList[1]

                sequences_array = np.array(sequences)
                targets_array = np.array(targets)
                
                # if training and test sets' expertise coincide, load test samples from the corresponding file
                # otherwise randomly select test_size = 2000 samples from the opposite expertise (same decision 
                # horizon and sampling time)
                if NameExertiseAndStep[key_expstep][:6] == NameExertiseAndStep[key_test][:6]: 

                    TrainTestSets_file = open(file_to_open,"rb")
                    indexes_data = pickle.load(TrainTestSets_file)
                    TrainTestSets_file.close()

                    test_index = indexes_data[1]

                    X_test = sequences_array[test_index]
                    y_test = targets_array[test_index] 

                else: 
                    print('Not matching!')

                    sss = StratifiedShuffleSplit(train_size=10, n_splits=1, 
                                         test_size=2000, random_state=0)  

                    for train_index, test_index in sss.split(sequences_array, targets_array):
                        X_train, X_test = sequences_array[train_index], sequences_array[test_index]
                        y_train, y_test = targets_array[train_index], targets_array[test_index]

                dummies_test = pd.get_dummies(y_test)

                test = X_test
                test_target = dummies_test.values
                
                # Load the trained ANN 
                model = load_model(file_model)
                
                # Use the trained ANN on the test set  
                test_preds = model.predict(test)

                # Compute metrics for the trained ANN 
                predicted_classes = np.argmax(test_preds,axis=1)
                expected_classes = np.argmax(test_target,axis=1)
                correct = metrics.accuracy_score(expected_classes,predicted_classes)

                print("Accuracy: %.2f%%" % (correct*100))

                precision_recall_f1 = metrics.precision_recall_fscore_support(expected_classes,predicted_classes)

                precision = 0
                recall = 0
                f1 = 0

                for i in range (5): 
                    precision = precision + precision_recall_f1[0][i]
                    recall = recall + precision_recall_f1[1][i]
                    f1 = f1 + precision_recall_f1[2][i]

                kappascore = metrics.cohen_kappa_score(expected_classes, predicted_classes)
                
#                 # Save performance metrics in .mat file to prepare MATLAB plots 
#                 matLabel = NameExertiseAndStep[key_expstep][:6] + "_" + NameExertiseAndStep[key_test] 
#                 + "_Horizon" + NameHorizon[key_horizon]

#                 matDict = {"accuracy_"+matLabel: correct*100, "precision_"+matLabel: precision*100 / 5, 
#                            "recall_"+matLabel : recall*100 / 5, "f1score_"+matLabel : f1*100 / 5, 
#                            "Kscore_"+matLabel : kappascore}

#                 savemat(matLabel+".mat", matDict)
#                 print(matLabel)

            else:
                print('------------------')

